In [31]:
import spacy
import en_core_web_sm
from spacy import displacy
import numpy as np
from IPython.core.debugger import set_trace
import os

from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifierCV, RidgeCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
import pandas as pd
from collections import Counter
from scipy.stats import wilcoxon
from IPython.core.debugger import set_trace
import pdb

np.random.seed(97)

save_dir = "features"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
nlp = spacy.load("en_core_web_lg")

In [3]:
labeled_words = pd.read_excel("word_ratings/WordSet1_Ratings.xlsx")

C:\Users\anike\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [16]:
words = labeled_words["Word"].to_numpy()
labels = labeled_words["Category"].to_numpy()

print("Number of words = {}".format(words.shape[0]))
print("Number of labels = {}".format(labels.shape[0]))
print("Number of unique labels = {}".format(np.unique(labels).shape[0]))

Number of words = 535
Number of labels = 535
Number of unique labels = 47


In [32]:
# get embeddings from spacy
spacy_embs = []
for word in words:
    word = nlp(word)
    spacy_embs.append(word.vector)
spacy_embs = np.array(spacy_embs)
print("Spacy embeddings shape = {}".format(spacy_embs.shape))

Spacy embeddings shape = (535, 300)


In [60]:
# fit RidgeCV classification model
clf = RidgeClassifierCV(alphas=np.logspace(-3, 6, num=20), scoring="balanced_accuracy")
clf.fit(spacy_embs, labels)

RidgeClassifierCV(alphas=array([1.00000000e-03, 2.97635144e-03, 8.85866790e-03, 2.63665090e-02,
       7.84759970e-02, 2.33572147e-01, 6.95192796e-01, 2.06913808e+00,
       6.15848211e+00, 1.83298071e+01, 5.45559478e+01, 1.62377674e+02,
       4.83293024e+02, 1.43844989e+03, 4.28133240e+03, 1.27427499e+04,
       3.79269019e+04, 1.12883789e+05, 3.35981829e+05, 1.00000000e+06]),
                  scoring='balanced_accuracy')

In [51]:
hp_text = open("chapter9.txt","r").readlines()
all_text = []
para = ""
for i in range(len(hp_text)):
    hp_text[i] = hp_text[i].strip()
    if hp_text[i]!="+":
        para += hp_text[i] + " "
    else:
        all_text.append(para)
        para = ""

In [61]:
all_vecs = []
for i in range(len(hp_text)):
    all_vecs.append([])

ind = 0
rem_len = len(hp_text[ind])
for para in all_text:
    proc = nlp(para)
    for word in proc:
        all_vecs[ind].append(word.vector)
        rem_len -= len(word.text)
        if rem_len <= 0:
            ind += 1
            while(ind < len(hp_text) and hp_text[ind] == "+"):
                ind += 1
            if ind < len(hp_text):
                rem_len = len(hp_text[ind]) + rem_len
c = [[hp_text[i],all_vecs[i]] for i in range(5176)]
for i, entry in enumerate(c):
    if c[i][0] == "+":
        c[i][1] = np.zeros(300)
    else:
        c[i][1] = np.mean(np.array(c[i][1]), axis=0).reshape(-1)
    if c[i][1].shape != (300,):
        pdb.set_trace()
    c[i][1] = clf.predict(c[i][1].reshape(1, -1))[0]

In [63]:
c

[['Harry', 'animal'],
 ['had', 'social action'],
 ['never', 'mental state'],
 ['believed', 'social action'],
 ['he', 'human'],
 ['would', 'locative action'],
 ['meet', 'social event'],
 ['a', 'human'],
 ['boy', 'human'],
 ['he', 'human'],
 ['hated', 'mental state'],
 ['more', 'multimodal property'],
 ['than', 'abstract construct'],
 ['Dudley,', 'plant'],
 ['but', 'cognitive entity'],
 ['that', 'abstract construct'],
 ['was', 'human'],
 ['before', 'time period'],
 ['he', 'human'],
 ['met', 'social action'],
 ['Draco', 'plant'],
 ['Malfoy.', 'plant'],
 ['Still,', 'social construct'],
 ['first-year', 'time period'],
 ['Gryffindors', 'social action'],
 ['only', 'abstract construct'],
 ['had', 'social action'],
 ['Potions', 'food'],
 ['with', 'social action'],
 ['the', 'abstract construct'],
 ['Slytherins,', 'emotion'],
 ['so', 'multimodal property'],
 ['they', 'locative action'],
 ["didn't", 'mental state'],
 ['have', 'social action'],
 ['to', 'locative action'],
 ['put', 'locative action'